In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.4.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [454 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [108 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [975 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,091 kB]
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:13 http://securi

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)


In [4]:
# 2. Create a temporary view of the DataFrame.

df.createOrReplaceTempView('home_sales')

In [5]:
df.show(5)

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
+--------------------+----------+----------+------+--------+---------+----------

In [6]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
query = """
SELECT YEAR(date) AS Year,
ROUND(AVG(price), 2) AS avg_price_4_bed
FROM home_sales
WHERE bedrooms = 4
GROUP BY YEAR(date)
ORDER BY YEAR(date);
"""
spark.sql(query).show()

+----+---------------+
|Year|avg_price_4_bed|
+----+---------------+
|2019|       300263.7|
|2020|      298353.78|
|2021|      301819.44|
|2022|      296363.88|
+----+---------------+



In [7]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?

query_2 = """

SELECT YEAR(date_built) as Year,
ROUND(AVG(price), 2) as avg_price_3_bed_3_bath
FROM home_sales
WHERE bedrooms = 4
AND bathrooms = 3
GROUP BY YEAR(date_built)
ORDER BY YEAR(date_built);
"""

spark.sql(query_2).show()

+----+----------------------+
|Year|avg_price_3_bed_3_bath|
+----+----------------------+
|2010|             287529.97|
|2011|             294656.14|
|2012|             289533.13|
|2013|             291870.89|
|2014|             289935.92|
|2015|              299517.5|
|2016|             289090.11|
|2017|             289100.17|
+----+----------------------+



In [9]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?

query_3 = """

SELECT YEAR(date_built) as Year,
ROUND(AVG(price), 2) as avg_price_3_bed_3_bath_2_floor_2000_sf_plus
FROM home_sales
WHERE bedrooms = 4
AND bathrooms = 3
AND floors = 2
AND sqft_living >= 2000
GROUP BY YEAR(date_built)
ORDER BY YEAR(date_built);
"""

spark.sql(query_3).show()

+----+-------------------------------------------+
|Year|avg_price_3_bed_3_bath_2_floor_2000_sf_plus|
+----+-------------------------------------------+
|2010|                                  290237.14|
|2011|                                  293021.78|
|2012|                                  290024.97|
|2013|                                  293916.19|
|2014|                                  306202.48|
|2015|                                  305769.02|
|2016|                                  298295.66|
|2017|                                  289215.12|
+----+-------------------------------------------+



In [13]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

# minor discrepancy with the challenge instructions:
# "What is the "view" rating for homes costing more than or equal to $350,000? Determine the run time for this query, and round off your answer to two decimal places."

start_time = time.time()

query_4 = """

SELECT view, AVG(price) AS average_price
FROM home_sales
WHERE price >= 350000
GROUP BY view
ORDER BY view DESC;

"""

spark.sql(query_4).show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+------------------+
|view|     average_price|
+----+------------------+
|  99|1061201.4193548388|
|  98|1053739.3333333333|
|  97| 1129040.148148148|
|  96|1017815.9166666666|
|  95|         1054325.6|
|  94|         1033536.2|
|  93|1026006.0606060605|
|  92| 970402.5476190476|
|  91| 1137372.731707317|
|  90|1062654.1621621621|
|   9| 401393.3370786517|
|  89|        1107839.15|
|  88|1031719.3548387097|
|  87|         1072285.2|
|  86|        1070444.25|
|  85|1056336.7435897435|
|  84|1117233.1290322582|
|  83|       1033965.925|
|  82|         1063498.0|
|  81|1053472.7878787878|
+----+------------------+
only showing top 20 rows

--- 0.6722743511199951 seconds ---


In [ ]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")

DataFrame[]

In [ ]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

False

In [14]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

spark.sql(query_4).show()

print("--- %s seconds ---" % (time.time() - start_time))


+----+------------------+
|view|     average_price|
+----+------------------+
|  99|1061201.4193548388|
|  98|1053739.3333333333|
|  97| 1129040.148148148|
|  96|1017815.9166666666|
|  95|         1054325.6|
|  94|         1033536.2|
|  93|1026006.0606060605|
|  92| 970402.5476190476|
|  91| 1137372.731707317|
|  90|1062654.1621621621|
|   9| 401393.3370786517|
|  89|        1107839.15|
|  88|1031719.3548387097|
|  87|         1072285.2|
|  86|        1070444.25|
|  85|1056336.7435897435|
|  84|1117233.1290322582|
|  83|       1033965.925|
|  82|         1063498.0|
|  81|1053472.7878787878|
+----+------------------+
only showing top 20 rows

--- 0.49858736991882324 seconds ---


In [16]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data

df.write.partitionBy("date_built").mode("overwrite").parquet("home_sales_partitioned")

In [17]:
# 11. Read the parquet formatted data.

df_p = spark.read.parquet("home_sales_partitioned")

In [18]:
# 12. Create a temporary table for the parquet data.

df_p.createOrReplaceTempView('home_sales_p')

In [19]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()

query_5 = """
SELECT view, AVG(price) AS average_price
FROM home_sales_p
WHERE price >= 350000
GROUP BY view
ORDER BY view DESC;
"""

spark.sql(query_5).show()


print("--- %s seconds ---" % (time.time() - start_time))

+----+------------------+
|view|     average_price|
+----+------------------+
|  99|1061201.4193548388|
|  98|1053739.3333333333|
|  97| 1129040.148148148|
|  96|1017815.9166666666|
|  95|         1054325.6|
|  94|         1033536.2|
|  93|1026006.0606060605|
|  92| 970402.5476190476|
|  91| 1137372.731707317|
|  90|1062654.1621621621|
|   9| 401393.3370786517|
|  89|        1107839.15|
|  88|1031719.3548387097|
|  87|         1072285.2|
|  86|        1070444.25|
|  85|1056336.7435897435|
|  84|1117233.1290322582|
|  83|       1033965.925|
|  82|         1063498.0|
|  81|1053472.7878787878|
+----+------------------+
only showing top 20 rows

--- 0.7130327224731445 seconds ---


In [20]:
# 14. Uncache the home_sales temporary table.

spark.sql("uncache table home_sales")

DataFrame[]

In [21]:
# 15. Check if the home_sales is no longer cached

spark.catalog.isCached('home_sales')

False